## Preprocessing: Clean data into JSON 
Source: https://towardsdatascience.com/a-complete-guide-to-an-interactive-geographical-map-using-python-f4c5197e23e0#_=_

In [57]:
import pandas as pd
import geopandas as gpd
import json

In [58]:
data = pd.read_csv('../data/clean_data.csv', index_col = 0)
data.head()

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website,population
0,Afghanistan,AFG,2021-02-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,38041754.0
1,Afghanistan,AFG,2021-02-23,0.0,0.0,0.0,1367.0,0.0,0.0,0.0,35.9342,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,38041754.0
2,Afghanistan,AFG,2021-02-24,0.0,0.0,0.0,1367.0,0.0,0.0,0.0,35.9342,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,38041754.0
3,Afghanistan,AFG,2021-02-25,0.0,0.0,0.0,1367.0,0.0,0.0,0.0,35.9342,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,38041754.0
4,Afghanistan,AFG,2021-02-26,0.0,0.0,0.0,1367.0,0.0,0.0,0.0,35.9342,Oxford/AstraZeneca,Government of Afghanistan,http://www.xinhuanet.com/english/asiapacific/2...,38041754.0


In [59]:
fully_vaccinations = data[['country', 'iso_code', 'date', 'people_fully_vaccinated', 'population']]

In [60]:
fully_vaccinations.loc[:, 'fully_vac_per_capita'] = fully_vaccinations.loc[:, 'people_fully_vaccinated']/fully_vaccinations.loc[:, 'population']
fully_vaccinations.loc[:, 'date'] = pd.to_datetime(fully_vaccinations.loc[:, 'date'], format='%Y-%m-%d')
fully_vaccinations = fully_vaccinations[['country', 'iso_code', 'date', 'fully_vac_per_capita']]
max_full_vaccinations = fully_vaccinations.groupby('country', 'iso_code')['fully_vac_per_capita'].max().sort_values()

C:\Users\frank\anaconda3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\frank\anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


ValueError: No axis named iso_code for object type DataFrame

In [61]:
fully_vaccinations.head()

,country,iso_code,date,fully_vac_per_capita
0,Afghanistan,AFG,2021-02-22,0.0
1,Afghanistan,AFG,2021-02-23,0.0
2,Afghanistan,AFG,2021-02-24,0.0
3,Afghanistan,AFG,2021-02-25,0.0
4,Afghanistan,AFG,2021-02-26,0.0


In [62]:
max_full_vaccinations = fully_vaccinations.groupby('iso_code')['fully_vac_per_capita'].max().sort_values()

In [63]:
fully_vaccinated_df=pd.DataFrame({'iso_code':max_full_vaccinations.index.to_list(), 'fully_vac_per_capita': max_full_vaccinations.to_list()})

Import geo-location data for world map drawing

In [64]:
shape = '../data/world_shape/ne_110m_admin_0_countries.shp'
gdf = gpd.read_file(shape)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf.columns = ['geo_country', 'country_code', 'geometry']
gdf = gdf[gdf['geo_country']!= 'Antarctica']

To prep for `Bokeh`, we need to have a final data format of GeoJSON

In [68]:
fully_geo_vaccinations = gdf.merge(fully_vaccinated_df, left_on = 'country_code', right_on = 'iso_code', how = 'left')
fully_vaccinations = fully_geo_vaccinations.to_json()

## Plotting using Bokeh 

### Fully Vaccinated World Map (Heat Map)

In [70]:
from bokeh.io import output_notebook, show, output_file, save
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, ColorBar, Title
from bokeh.palettes import RdYlGn
from bokeh.transform import linear_cmap
from datetime import datetime

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = fully_vaccinations)

#Define a continuous multi-hue color palette. Reverse color order so that we go from red to green.
palette = RdYlGn[11]

palette = palette[::-1]
mapper = linear_cmap(field_name='fully_vac_per_capita', palette=palette ,low=0 ,high=0.5)
#Create color bar. 
color_bar = ColorBar(color_mapper=mapper['transform'], label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (270,0), orientation = 'horizontal')

#Define hover tool display
TOOLTIPS = [
    ("country", "@geo_country"),
    ("value", "@fully_vac_per_capita{0,0.0000}"),
]




#Create figure object.
p = figure(title = 'Road to COVID-19 Immunity through Vaccination', plot_height = 700 , plot_width = 1100, tools = 'hover',
          tooltips = TOOLTIPS)

#Style Grid/Axes/Title
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.outline_line_color = None
p.axis.visible = False
p.title.align = "center"
p.title.text_font_size = '20pt'

#Add map to figure 
p.patches('xs','ys', source = geosource,fill_color = mapper,
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
p.add_layout(color_bar, 'below')

#Add notes below graph
p.add_layout(Title(text="This world map shows the proportion of the population of countries around the world that has been fully vaccinated", 
                   align="center", text_font_size = '10pt'), "above")
p.add_layout(Title(text="Source: Our World in Data, Kaggle", align="center", text_alpha = 0.5), "below")

current_date = datetime.today().strftime('%Y-%m-%d')
update_text = "Last Updated on " + str(current_date)
p.add_layout(Title(text=update_text, align="center", text_alpha = 0.5), "below")
#Display figure inline in Jupyter Notebook.
output_notebook()
#Display figure.
output_file('fully_vac_map.html')
save(p)

Loading BokehJS ...

'C:\\Users\\frank\\OneDrive\\Documents\\Data Projects\\covid-19-vaccines\\scripts\\fully_vac_map.html'